In [ ]:
from connector_pendulum import connector_pendulum

solver_indices = [5]
n_psi_trains = [50, 75]

for index in solver_indices:
    for psi_train in n_psi_trains:
        connector_pendulum(psi_train, index)

In [ ]:
# ## Results from solver4
# evalues = solver.eigenvalues
# N_dict = np.shape(evalues)[0]
# # Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])
# Koopman_matrix_K = solver.K.numpy()
# Psi_X = solver.get_Psi_X().numpy()
# Psi_Y = solver.get_Psi_Y().numpy()
# # Assuming psi_x is your Psi_X matrix as a NumPy array
# Psi_X_U, _, _ = np.linalg.svd(Psi_X/np.sqrt(Psi_X.shape[0]), full_matrices=False)
# Psi_Y_U, _, _ = np.linalg.svd(Psi_Y/np.sqrt(Psi_Y.shape[0]), full_matrices=False)
# # Calculating the QR decomposition
# Psi_X_Q, _ = np.linalg.qr(Psi_X)
# Psi_Y_Q, _ = np.linalg.qr(Psi_Y)

# print("Check if Psi_X_U is orthonormal: ", np.allclose(np.dot(Psi_X_U.T, Psi_X_U), np.eye(Psi_X_U.shape[1])))
# print("Check if Psi_X_Q is orthonormal: ", np.allclose(np.dot(Psi_X_Q.T, Psi_X_Q), np.eye(Psi_X_Q.shape[1])))

# resDMD_DL_outputs4 = {
#     'resDMD_DL_outputs': {        
#         'evalues': evalues,
#         'N_dict': N_dict,
#         'Psi_X': Psi_X,
#         'Psi_Y': Psi_Y,
#         'Psi_X_U': Psi_X_U,
#         'Psi_Y_U': Psi_Y_U,
#         'Psi_X_Q': Psi_X_Q,
#         'Psi_Y_Q': Psi_Y_Q,
#         'K': Koopman_matrix_K,  # Add the Koopman matrix K to the results
#     }
# }

# # np.save("solver_4_outputs.npy", resDMD_DL_outputs4, allow_pickle=True)
# save_path = 'D:\\Residual-Dynamic-Mode-Decomposition-main\\Examples_gallery_1\\ResDMD_datasets'  # Ensure this is a directory
# nd = resDMD_DL_outputs4['resDMD_DL_outputs']['evalues'].shape[0]
# filename = f'solver4_outputs_{nd}basis.mat'
# scipy.io.savemat(save_path + filename, resDMD_DL_outputs4)

In [ ]:
# ## Plot eigenvalues
# import matplotlib.pyplot as plt

# # Plot the unit circle for reference
# theta = np.linspace(0, 2*np.pi, 100)
# plt.plot(np.cos(theta), np.sin(theta), linestyle='dashed')

# # Plot the real and imaginary parts of the eigenvalues
# plt.scatter(evalues.real, evalues.imag, color='red', label='Eigenvalues')

# # Set equal scaling for both axes
# plt.axis('equal')

# # Set the labels
# plt.xlabel('Real part')
# plt.ylabel('Imaginary part')
# plt.title('Eigenvalues in the Complex Plane')

# # Enable the legend
# plt.legend()

# # Show the plot
# plt.show()